## Trained Embedded Layer
- created: 23 MArch 2020

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from os.path import join
import tensorflow as tf
from tensorflow import keras
from joblib import dump, load
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
from tensorflow.keras import callbacks
from time import time
from datetime import datetime
import shutil 
import argparse
from functools import partial
import pickle
from IPython.display import Audio

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Reshape, GRU, InputLayer, AlphaDropout, Activation, BatchNormalization, Dropout, Flatten, Dense, Bidirectional, LSTM, Conv1D, SpatialDropout1D, Concatenate, Multiply, Add

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import OneHotEncoder

In [2]:
np_load_old = partial(np.load)
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)
tf.random.set_seed(42)

## Load Data 

In [7]:
def prep_data(data_dir,prep_folder):

    features = np.load(join(data_dir,prep_folder,'features.npy'))
    
    with open(join(data_dir,'syllable_name.txt')) as f:
        label = np.array([char for line in f for word in line.split(',') for char in word.strip()])
        
    print('Predict Data {} {}'.format(features.shape, label.shape))
    return features, label

In [76]:
DATA_PATH = '../../data/d_records/record_total_data/'
PREP_FOLDER = 'prep_data'
features, label = prep_data(DATA_PATH, PREP_FOLDER)

Predict Data (3078, 23, 39) (3078,)


In [77]:
set(label)

{'7', 'E', 'M', 'O', 'a', 'e', 'i', 'o', 'u'}

In [78]:
label.shape

(3078,)

### One-hot label 

In [79]:
label=label.reshape(-1,1)

enc = OneHotEncoder(handle_unknown = 'ignore')
enc.fit(label)

label_oe = enc.transform(label).toarray()

dump(enc, 'onehot_pred.joblib')

['onehot_pred.joblib']

In [80]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, label_oe, test_size=0.2)

### Defined (Between) Embedding Model

In [146]:
PHONETIC_OUTPUT = len(['7', 'E', 'M', 'O', 'a', 'e', 'i', 'o', 'u'])
EMBEDDED_FEATURE = 64

pLSTM = partial(LSTM, kernel_initializer='he_uniform', return_sequences=True)

def embeded_model(input_shape_1,input_shape_2):
    inputs = keras.Input(shape=(input_shape_1,input_shape_2))
    x = pLSTM(EMBEDDED_FEATURE)(inputs)
    x = Dropout(rate=0.5)(x)
    x = pLSTM(EMBEDDED_FEATURE, return_sequences=False)(x)
    x = Dropout(rate=0.5)(x)
    outputs = Dense(PHONETIC_OUTPUT, activation='softmax', kernel_initializer='he_uniform')(x)
    return keras.Model(inputs=inputs, outputs=outputs)

In [147]:
embedded = embeded_model(23, 39)
embedded.summary()

Model: "model_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_18 (InputLayer)        [(None, 23, 39)]          0         
_________________________________________________________________
lstm_39 (LSTM)               (None, 23, 64)            26624     
_________________________________________________________________
dropout_37 (Dropout)         (None, 23, 64)            0         
_________________________________________________________________
lstm_40 (LSTM)               (None, 64)                33024     
_________________________________________________________________
dropout_38 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_16 (Dense)             (None, 9)                 585       
Total params: 60,233
Trainable params: 60,233
Non-trainable params: 0
______________________________________________________

### Compile Model 

In [148]:
embedded.compile(optimizer='adam',loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),metrics=['accuracy'])

### Train Model 

In [149]:
PATIENCE = 15
BATCH_SIZE = 128
EPOCHS = 200

early = callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=PATIENCE, verbose=1, mode='min', baseline=None, restore_best_weights=False)
callback_list = [early]

In [150]:
history = embedded.fit(X_train,y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    verbose=2,
    validation_split=0.2,
    callbacks=callback_list)

Train on 1969 samples, validate on 493 samples
Epoch 1/200
1969/1969 - 4s - loss: 2.1652 - accuracy: 0.2331 - val_loss: 2.0904 - val_accuracy: 0.5132
Epoch 2/200
1969/1969 - 0s - loss: 2.0553 - accuracy: 0.4794 - val_loss: 1.9252 - val_accuracy: 0.6795
Epoch 3/200
1969/1969 - 0s - loss: 1.9155 - accuracy: 0.6369 - val_loss: 1.7628 - val_accuracy: 0.7688
Epoch 4/200
1969/1969 - 0s - loss: 1.7694 - accuracy: 0.7669 - val_loss: 1.6248 - val_accuracy: 0.8641
Epoch 5/200
1969/1969 - 0s - loss: 1.6480 - accuracy: 0.8532 - val_loss: 1.5439 - val_accuracy: 0.8783
Epoch 6/200
1969/1969 - 0s - loss: 1.5547 - accuracy: 0.9035 - val_loss: 1.5035 - val_accuracy: 0.8945
Epoch 7/200
1969/1969 - 0s - loss: 1.5017 - accuracy: 0.9253 - val_loss: 1.4857 - val_accuracy: 0.9026
Epoch 8/200
1969/1969 - 0s - loss: 1.4768 - accuracy: 0.9375 - val_loss: 1.4724 - val_accuracy: 0.9108
Epoch 9/200
1969/1969 - 0s - loss: 1.4576 - accuracy: 0.9436 - val_loss: 1.4667 - val_accuracy: 0.9087
Epoch 10/200
1969/1969 - 0

In [151]:
embedded.save('pred_vowel.hdf5')

In [14]:
def evaluate_model(y_pred, label):
    y_pred = enc.inverse_transform(y_pred)
    label = enc.inverse_transform(label)
#     print('Confusion Matrix')
    print(confusion_matrix(label, y_pred))
    print('Classification Report')
    print(classification_report(label, y_pred))

In [153]:
pred = embedded.predict(X_test)
evaluate_model(pred, y_test)

[[71  4  1  1  0  2  0  0  0]
 [ 1 70  0  0  0  1  0  0  0]
 [ 0  0 58  0  0  2  1  0  0]
 [ 0  0  0 73  0  0  0  1  0]
 [ 0  0  0  1 53  0  0  0  0]
 [ 0  0  0  0  0 64  3  0  0]
 [ 0  0  0  0  0  1 66  0  0]
 [ 0  0  0  0  0  0  0 64  1]
 [ 0  0  0  0  0  0  1  2 74]]
Classification Report
              precision    recall  f1-score   support

           7       0.99      0.90      0.94        79
           E       0.95      0.97      0.96        72
           M       0.98      0.95      0.97        61
           O       0.97      0.99      0.98        74
           a       1.00      0.98      0.99        54
           e       0.91      0.96      0.93        67
           i       0.93      0.99      0.96        67
           o       0.96      0.98      0.97        65
           u       0.99      0.96      0.97        77

    accuracy                           0.96       616
   macro avg       0.96      0.96      0.96       616
weighted avg       0.96      0.96      0.96       616



### Train using all data 

In [154]:
embedded = embeded_model(23, 39)
embedded.compile(optimizer='adam',loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
history = embedded.fit(features,label_oe,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    verbose=2,
    validation_split=0.2,
    callbacks=callback_list)

Train on 1555 samples, validate on 389 samples
Epoch 1/200
1555/1555 - 4s - loss: 2.1538 - accuracy: 0.2521 - val_loss: 2.0380 - val_accuracy: 0.5835
Epoch 2/200
1555/1555 - 0s - loss: 2.0234 - accuracy: 0.5325 - val_loss: 1.8814 - val_accuracy: 0.6221
Epoch 3/200
1555/1555 - 0s - loss: 1.9043 - accuracy: 0.5968 - val_loss: 1.7897 - val_accuracy: 0.6555
Epoch 4/200
1555/1555 - 0s - loss: 1.8174 - accuracy: 0.6566 - val_loss: 1.7450 - val_accuracy: 0.6761
Epoch 5/200
1555/1555 - 0s - loss: 1.7468 - accuracy: 0.7093 - val_loss: 1.7151 - val_accuracy: 0.6889
Epoch 6/200
1555/1555 - 0s - loss: 1.7013 - accuracy: 0.7305 - val_loss: 1.6921 - val_accuracy: 0.6915
Epoch 7/200
1555/1555 - 0s - loss: 1.6727 - accuracy: 0.7395 - val_loss: 1.6721 - val_accuracy: 0.7147
Epoch 8/200
1555/1555 - 0s - loss: 1.6512 - accuracy: 0.7627 - val_loss: 1.6615 - val_accuracy: 0.7224
Epoch 9/200
1555/1555 - 0s - loss: 1.6403 - accuracy: 0.7640 - val_loss: 1.6350 - val_accuracy: 0.7378
Epoch 10/200
1555/1555 - 0

In [155]:
embedded.save('pred_vowel_all_data.hdf5')

## Load Model

In [10]:
embedded = models.load_model('pred_vowel_all_data.hdf5')
enc = load('onehot_pred.joblib')

## Predict 

In [11]:
DATA_PATH = '../../data/inversion_81/'
PREP_FOLDER = 'prep_data'
features, label = prep_data(DATA_PATH, PREP_FOLDER)

Predict Data (1944, 23, 39) (1944,)


In [12]:
label= label.reshape(-1,1)
label_oe = enc.transform(label).toarray()

In [15]:
pred = embedded.predict(features)
evaluate_model(pred, label_oe)

[[118  25  22   9   1  29   1   7   4]
 [ 17 149   4   4  20  21   0   0   1]
 [ 50   5  88   0   2  64   4   1   2]
 [ 20  12   3 113  19   4   0  39   6]
 [ 14  52   8  15 122   4   0   1   0]
 [  9   6   4   0   0 166  30   1   0]
 [ 11   1  19   1   4  42 127   1  10]
 [ 23   0   1  25   1   1   0  92  73]
 [ 81   1  11   7   4  15   2  24  71]]
Classification Report
              precision    recall  f1-score   support

           7       0.34      0.55      0.42       216
           E       0.59      0.69      0.64       216
           M       0.55      0.41      0.47       216
           O       0.65      0.52      0.58       216
           a       0.71      0.56      0.63       216
           e       0.48      0.77      0.59       216
           i       0.77      0.59      0.67       216
           o       0.55      0.43      0.48       216
           u       0.43      0.33      0.37       216

    accuracy                           0.54      1944
   macro avg       0.56      0

### Predict Data 27

In [159]:
DATA_PATH = '../../data/inversion_27/'
PREP_FOLDER = 'prep_data'
features, label = prep_data(DATA_PATH, PREP_FOLDER)

Predict Data (1944, 23, 39) (1944,)


In [160]:
label= label.reshape(-1,1)
label_oe = enc.transform(label).toarray()

In [161]:
pred = embedded.predict(features)
evaluate_model(pred, label_oe)

[[143  26  16   4   2  15   0   7   3]
 [  7 193   0   1  10   3   0   2   0]
 [ 10   0 163   0   0  24  15   1   3]
 [  4   2   0 147  36   0   0  25   2]
 [  1  18   2  28 165   2   0   0   0]
 [  2   7   3   0   0 194  10   0   0]
 [  0   0  13   0   0  34 168   0   1]
 [  8   0   1   8   1   1   0 117  80]
 [  7   0  21   0   0   4  18  17 149]]
Classification Report
              precision    recall  f1-score   support

           7       0.79      0.66      0.72       216
           E       0.78      0.89      0.84       216
           M       0.74      0.75      0.75       216
           O       0.78      0.68      0.73       216
           a       0.77      0.76      0.77       216
           e       0.70      0.90      0.79       216
           i       0.80      0.78      0.79       216
           o       0.69      0.54      0.61       216
           u       0.63      0.69      0.66       216

    accuracy                           0.74      1944
   macro avg       0.74      0

In [162]:
class_pred = np.argmax(pred, axis=1)
class_actual = np.argmax(label_oe, axis=1)

In [163]:
class_pred == class_actual

array([ True,  True, False, ..., False,  True, False])

### Predict Data 64 
- Conv Bilstm with symbolic map

In [16]:
DATA_PATH = '../../data/inversion_64/'
PREP_FOLDER = 'prep_data'
features, label = prep_data(DATA_PATH, PREP_FOLDER)

Predict Data (1944, 23, 39) (1944,)


In [17]:
label= label.reshape(-1,1)
label_oe = enc.transform(label).toarray()

In [18]:
pred = embedded.predict(features)
evaluate_model(pred, label_oe)

[[156  26  14   3   0   9   1   3   4]
 [ 15 183   0   4  11   3   0   0   0]
 [ 10   2 151   0   1  23  19   0  10]
 [  3  12   0 134  50   0   0  16   1]
 [  6  13   1  23 172   0   0   1   0]
 [  1   9   5   0   0 200   1   0   0]
 [  1   0  13   0   0  38 163   0   1]
 [  7   5   5  11   2   8   1 111  66]
 [  5   0   8   0   2   4  11   7 179]]
Classification Report
              precision    recall  f1-score   support

           7       0.76      0.72      0.74       216
           E       0.73      0.85      0.79       216
           M       0.77      0.70      0.73       216
           O       0.77      0.62      0.69       216
           a       0.72      0.80      0.76       216
           e       0.70      0.93      0.80       216
           i       0.83      0.75      0.79       216
           o       0.80      0.51      0.63       216
           u       0.69      0.83      0.75       216

    accuracy                           0.75      1944
   macro avg       0.75      0

In [162]:
class_pred = np.argmax(pred, axis=1)
class_actual = np.argmax(label_oe, axis=1)

In [163]:
class_pred == class_actual

array([ True,  True, False, ..., False,  True, False])

### Predict Data 31

In [164]:
DATA_PATH = '../../data/inversion_31/'
PREP_FOLDER = 'prep_data'
features, label = prep_data(DATA_PATH, PREP_FOLDER)

Predict Data (1944, 23, 39) (1944,)


In [165]:
label= label.reshape(-1,1)
label_oe = enc.transform(label).toarray()

In [166]:
pred = embedded.predict(features)
evaluate_model(pred, label_oe)

[[177  16  16   0   0   5   1   1   0]
 [  6 196   1   0   5   8   0   0   0]
 [  2   1 164   0   0  29  18   0   2]
 [  2   6   2 163  29   0   0  14   0]
 [  1  10   4  12 188   1   0   0   0]
 [  0   2   2   0   0 211   1   0   0]
 [  0   0   6   0   0  10 197   0   3]
 [ 10   2   1   7   0   5   0 145  46]
 [  4   2  20   0   0   5  15  12 158]]
Classification Report
              precision    recall  f1-score   support

           7       0.88      0.82      0.85       216
           E       0.83      0.91      0.87       216
           M       0.76      0.76      0.76       216
           O       0.90      0.75      0.82       216
           a       0.85      0.87      0.86       216
           e       0.77      0.98      0.86       216
           i       0.85      0.91      0.88       216
           o       0.84      0.67      0.75       216
           u       0.76      0.73      0.74       216

    accuracy                           0.82      1944
   macro avg       0.83      0